# Predicting the Price of the S&P500 Index with Linear Regression

In this project, we worked with data from the S&P500 Index. The S&P500 is a stock market index. We used historical data on the price of the S&P500 Index to make predictions about future prices.

We'll be working with a csv file containing index prices. Each row in the file contains a daily record of the price of the S&P500 Index from 1950 to 2015. The dataset is stored in sphist.csv.

The columns of the dataset are:

* Date -- The date of the record.
* Open -- The opening price of the day (when trading starts).
* High -- The highest trade price during the day.
* Low -- The lowest trade price during the day.
* Close -- The closing price for the day (when trading is finished).
* Volume -- The number of shares traded.
* Adj Close -- The daily closing price, adjusted retroactively to include any corporate actions.

We'll be using this dataset to develop a predictive model. You'll train the model with data from 1950-2012 and try to make predictions from 2013-2015.

Note: You shouldn't make trades with any models developed in this lesson. Trading stocks has risks and nothing in this lesson constitutes stock trading advice.

In [35]:
import pandas as pd
from pandas import DataFrame
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [36]:
#Read in the data set
df = pd.read_csv(r'C:\Users\henye\Downloads\sphist.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [37]:
#Sort by the date column in ascending order
df = df.sort_values("Date", ascending=True)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000,17.08


Add features that could be helpful for machine learning

We will use Pandas Time series tools: Rolling function to set the window equal to the past 5 trading days to compute the indicators. 
This adds in NaN values for any row where there aren't enough historical trading days to do the computation.

Note: We could use also indicators with 30 or 365 days of historical data. For the last case, since our dataset starts on 1950-01-03, for any rows that fall before 1951-01-03 we wouldn't have enough historical data to compute all the indicators. For this reason we would need to remove these rows before splitting the data as followed:

removing any rows from df that occur before 1951-01-03
df = df[df['Date'] >= datetime(year=1951, month=1, day=3)]
df.dropna(axis=0)
df.head()

In [38]:
df['5 Days Open'] = df['Open'].rolling(window=5).mean()
df['5 Days High'] = df['High'].rolling(window=5).mean()
df['5 Days Low'] = df['Low'].rolling(window=5).mean()
df['5 Days Volume'] = df['Volume'].rolling(window=5).mean()
df['Year'] = df['Date'].apply(lambda x: x.year)

df.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,5 Days Open,5 Days High,5 Days Low,5 Days Volume,Year
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000,16.660000,NaN,NaN,NaN,NaN,1950
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000,16.850000,NaN,NaN,NaN,NaN,1950
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000,16.930000,NaN,NaN,NaN,NaN,1950
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000,16.980000,NaN,NaN,NaN,NaN,1950
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000,17.080000,16.900,16.900,16.900,2046000.0,1950
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000,17.030001,16.974,16.974,16.974,2226000.0,1950
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000,17.090000,17.022,17.022,17.022,2374000.0,1950
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000,16.760000,16.988,16.988,16.988,2458000.0,1950
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000,16.670000,16.926,16.926,16.926,2722000.0,1950
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000,16.719999,16.854,16.854,16.854,2510000.0,1950


In [39]:
#Adding Day of week column and set it to categorical
df['DOW'] = df['Date'].apply(lambda x: x.weekday())

#Returning the dummy columns for the new DOW
dow_df = pd.get_dummies(df['DOW'])
df = pd.concat([df, dow_df], axis=1)
df = df.drop(['DOW'], axis=1)

Because 'rolling' method above include the current date to compute, the current date include future knowledge, which means that the prediction will look not good in real world , so we shift all the values forward one day.
This way the values will be the mean of the previous 5 days

In [40]:
df['5 Days Open'] = df['5 Days Open'].shift(1)
df['5 Days High'] = df['5 Days High'].shift(1)
df['5 Days Low'] = df['5 Days Low'].shift(1)
df['5 Days Volume'] = df['5 Days Volume'].shift(1)

#Let's remove also the rows Nan Values in our df
df = df.dropna(axis=0)


df.head(6)

,Date,Open,High,Low,Close,Volume,Adj Close,5 Days Open,5 Days High,5 Days Low,5 Days Volume,Year,0,1,2,3,4
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000,17.030001,16.900,16.900,16.900,2046000.0,1950,0,1,0,0,0
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000,17.090000,16.974,16.974,16.974,2226000.0,1950,0,0,1,0,0
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000,16.760000,17.022,17.022,17.022,2374000.0,1950,0,0,0,1,0
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000,16.670000,16.988,16.988,16.988,2458000.0,1950,0,0,0,0,1
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000,16.719999,16.926,16.926,16.926,2722000.0,1950,1,0,0,0,0
16579,1950-01-17,16.860001,16.860001,16.860001,16.860001,1790000,16.860001,16.854,16.854,16.854,2510000.0,1950,0,1,0,0,0


# Linear Regression

In [41]:
#Split dataset into train set and test set.
train_df = df[df['Date'] < datetime(year=2013, month=1, day=1)]
test_df = df[df['Date'] >= datetime(year=2013, month=1, day=1)]

#Feature columns
train_colums = ['5 Days Open', '5 Days Volume', '5 Days High', '5 Days Low', 'Year', 0, 1, 2, 3, 4]

In [42]:
# Perform linear regression.
lr = LinearRegression()
lr.fit(train_df[train_colums], train_df['Close'])
prediction = lr.predict(test_df[train_colums])

# Model Evaluation

In [43]:
#Comparing the predicted price againts the real prices in our data bases

test_df_copy = test_df.copy()
test_df_copy['Predicted close price'] = prediction
test_df_copy['Difference'] = (test_df_copy['Close'] - test_df_copy['Predicted close price'])
test_df_copy['Accuracy(%)'] = (1-(abs(test_df_copy['Close'] - test_df_copy['Predicted close price']) / test_df_copy['Close']
)) * 100
test_df_copy = DataFrame(test_df_copy[['Date','Close','Predicted close price','Difference', 'Accuracy(%)']])

print(test_df_copy)

          Date        Close  Predicted close price  Difference  Accuracy(%)
738 2013-01-02  1462.420044            1409.284048   53.135996    96.366571
737 2013-01-03  1459.369995            1429.894239   29.475756    97.980241
736 2013-01-04  1466.469971            1438.453330   28.016641    98.089518
735 2013-01-07  1461.890015            1456.819407    5.070608    99.653147
734 2013-01-08  1457.150024            1469.353293  -12.203269    99.162525
..         ...          ...                    ...         ...          ...
4   2015-12-01  2102.629883            2081.036097   21.593786    98.973011
3   2015-12-02  2079.510010            2090.819048  -11.309038    99.456168
2   2015-12-03  2049.620117            2085.927833  -36.307716    98.228564
1   2015-12-04  2091.689941            2069.194303   22.495638    98.924523
0   2015-12-07  2077.070068            2083.777398   -6.707330    99.677077

[739 rows x 5 columns]


In [44]:
# Error metrics.
mse = mean_squared_error(test_df['Close'], prediction)
rmse = np.sqrt(mse)

print('mse:',mse)
print('rmse:',rmse)
print('-'*60)
print('The S&P500 Index')

display(test_df_copy.sort_values(by=['Date'], ascending=False).head(20))

mse: 378.84390622601336
rmse: 19.463912921764045
------------------------------------------------------------
The S&P500 Index


,Date,Close,Predicted close price,Difference,Accuracy(%)
0,2015-12-07,2077.070068,2083.777398,-6.707330,99.677077
1,2015-12-04,2091.689941,2069.194303,22.495638,98.924523
2,2015-12-03,2049.620117,2085.927833,-36.307716,98.228564
3,2015-12-02,2079.510010,2090.819048,-11.309038,99.456168
4,2015-12-01,2102.629883,2081.036097,21.593786,98.973011
5,2015-11-30,2080.409912,2089.935869,-9.525957,99.542112
6,2015-11-27,2090.110107,2087.940141,2.169966,99.896179
7,2015-11-25,2088.870117,2097.578008,-8.707891,99.583129
8,2015-11-24,2089.139893,2095.837962,-6.698069,99.679386
9,2015-11-23,2086.590088,2097.873983,-11.283895,99.459218
